In [3]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from pandas import concat

In [2]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    From Jason Brownlee
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
        
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
            
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    
    return agg

In [4]:
values = [x for x in np.arange(10)]
data = series_to_supervised(values,2,1)
print(data)

   var1(t-2)  var1(t-1)  var1(t)
2        0.0        1.0        2
3        1.0        2.0        3
4        2.0        3.0        4
5        3.0        4.0        5
6        4.0        5.0        6
7        5.0        6.0        7
8        6.0        7.0        8
9        7.0        8.0        9


In [5]:
def plot_feature_importances(model):
    n_features = X_train.shape[1]
    plt.barh(np.arange(n_features), model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features), X_train.columns)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    plt.ylim(-1, n_features) 

In [6]:
def printConfusionMatrix(y_test, y_pred, imagename):
    confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
    plt.matshow(confusion_matrix)
    plt.title('Confusion matrix')
    plt.colorbar()
    plt.ylabel('Churned')
    plt.xlabel('Predicted')
    plt.savefig(imagename)
    plt.show()
    print(confusion_matrix)

In [12]:
# Model evaluation 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, average_precision_score
def model_evaluation(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:,1]
    print('Test accuracy was {:.2f}', accuracy_score(y_test, y_pred))
    print('Test average precision was {:.2f}', average_precision_score(y_test, y_pred_prob))
    print("\n")
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    print('True Negative = {:.2f}\tFalse Positive = {:.2f}\nFalse Negative = {:.2f}\tTrue Positive={:.2f}'
          .format(tn,fp,fn,tp))
    print("\n")
    print(classification_report(y_test, y_pred))